# Superposed Epoch Analysis 20CR model runs
- opens all the 20CR data on precipitation pr and surface temperature tmp
- calculates monthly anomalies (based on 5-yr climatology) for 4 spatial regions (global, SH, Aus, E Aus) and 5 eruptions
- calculates percentiles (10, 90) to show model spread
- superposes the time-series for 5 eruptions plus mmm for tmp and pr and different regions (superposed epoch analysis)

In [1]:
import xarray as xr, matplotlib.pyplot as plt
from importlib import reload # need to use this if I edit a function file
import os
import numpy as np
import pandas as pd
import cartopy.crs as ccrs # to add in continents and change map projections 
from matplotlib.colors import LinearSegmentedColormap # to change colour bar????
import seaborn as sns
import dask.diagnostics # dask allows you to check how long something is taking to load

In [2]:
# import custom functions
import sys 
sys.path.append('/home/563/kb6999/Functions') # use this if the function file is in a different directory to the notebook

import frequently_used_functions as func
import plotting_functions as fplot
import model_functions as funcM
import reanalysis_functions as funcR

In [3]:
import dask.distributed
import tempfile

tempdir = tempfile.TemporaryDirectory("dask-worker-space")
dask.distributed.Client(local_directory=tempdir.name, memory_limit='16gb')

Client Scheduler: tcp://127.0.0.1:38239 Dashboard: http://127.0.0.1:8787/status,Cluster Workers: 2 Cores: 2 Memory: 11.23 GiB


In [4]:
path = '/g/data/w48/kb6999/Reanalysis/20CR_SEA_data_for_plots/'
# choose which variable to save as netcdfs
var = 'tmp'
# var = 'pr'

In [5]:
# create a list of member names excluding member 70 cos that file is problematic 
members = [*range(1,70),*range(71,81)]

In [6]:
# create a list of the paths and then open all members 
if var == 'tmp':
    print('var=tmp')
    tmp_paths = [f"/g/data/w48/kb6999/Reanalysis/20CR_TMP_raw_members/R_raw_Glob_TMP{m:02d}.nc" for m in members]
    ds_tmp = xr.open_mfdataset(tmp_paths, combine='nested', concat_dim='member', chunks={'time': 200})
    ds_tmp.coords['member'] = members
    reanal = xr.Dataset({'tmp': ds_tmp.TMP})


if var == 'pr':
    print(var=pr)
    pr_paths = [f"/g/data/w48/kb6999/Reanalysis/20CR_PRATE_raw_members/R_anom_Glob_PRATE{m:02d}.nc" for m in members]
    ds_pr = xr.open_mfdataset(pr_paths, combine='nested', concat_dim='member', chunks={'time': 200})
    ds_pr.coords['member'] = members
    reanal = xr.Dataset({'pr': ds_pr.PRATE})


var=tmp


In [7]:
# area weighting 
reanal_w = reanal*np.cos(reanal.lat*(np.pi/180))
reanal_w

,Array,Chunk
Bytes,76.84 GiB,100.00 MiB
Shape,"(79, 1992, 256, 512)","(1, 200, 256, 512)"
Count,3240 Tasks,790 Chunks
Type,float32,numpy.ndarray


Right now I have all the members stored in one array so from hereafter I can calculate anomalies etc.  
## Dates and volcano names

In [8]:
# eruption date
e_date = pd.to_datetime(['1883-08','1902-10', '1963-03', '1982-04', '1991-06'])
# e_pre5yrs = ['1878-08','1897-10', '1958-03', '1977-04', '1986-06']
# e_pre3mon = ['1883-05','1902-7', '1962-12', '1982-01', '1991-03']
# e_post5yrs = ['1888-08','1907-10', '1968-03', '1987-04', '1996-06']

In [9]:
# save arrays of times relative to each eruption
e_pre5yrs = e_date - 60 * pd.offsets.MonthBegin()
e_pre3mon = e_date - 3 * pd.offsets.MonthBegin()
e_post5yrs = e_date + 61 * pd.offsets.MonthBegin()

In [10]:
print(e_pre5yrs[1], e_pre3mon[0], e_post5yrs)

1897-10-01 00:00:00 1883-05-01 00:00:00 DatetimeIndex(['1888-09-01', '1907-11-01', '1968-04-01', '1987-05-01',
               '1996-07-01'],
              dtype='datetime64[ns]', freq=None)


In [11]:
# eruption names
volcanoes = ['Krakatoa', 'Santa Maria','Agung','El Chichon', 'Pinatubo']

## Select out regions and eruption dates

In [12]:
# select out hte 10 year time frame around each eruption (5yrs before and after)
# tmp
Krakatoa = reanal_w.sel(time = slice(e_pre5yrs[0], e_post5yrs[0]))
SantaMaria = reanal_w.sel(time = slice(e_pre5yrs[1], e_post5yrs[1]))
Agung = reanal_w.sel(time = slice(e_pre5yrs[2], e_post5yrs[2]))
ElChicon = reanal_w.sel(time = slice(e_pre5yrs[3], e_post5yrs[3]))
Pinatubo = reanal_w.sel(time = slice(e_pre5yrs[4], e_post5yrs[4]))

## Anomalies 

In [14]:
# use functions to calculate the monthly anomalies for the globe
K_mon_anom_Glob = func.monthly_anomaly(Krakatoa, e_pre5yrs[0], e_pre3mon[0])
S_mon_anom_Glob = func.monthly_anomaly(SantaMaria, e_pre5yrs[1], e_pre3mon[1])
A_mon_anom_Glob = func.monthly_anomaly(Agung, e_pre5yrs[2], e_pre3mon[2])
E_mon_anom_Glob = func.monthly_anomaly(ElChicon, e_pre5yrs[3], e_pre3mon[3])
P_mon_anom_Glob = func.monthly_anomaly(Pinatubo, e_pre5yrs[4], e_pre3mon[4])
P_mon_anom_Glob

/g/data3/hh5/public/apps/miniconda3/envs/analysis3-21.04/lib/python3.8/site-packages/xarray/core/indexing.py:1379: PerformanceWarning: Slicing with an out-of-order index is generating 11 times more chunks
  return self.array[key]


,Array,Chunk
Bytes,4.67 GiB,512.00 kiB
Shape,"(79, 121, 256, 512)","(1, 1, 256, 512)"
Count,23385 Tasks,9559 Chunks
Type,float32,numpy.ndarray


## save off anomalies as netcdfs 

### don't need to run the next two cos i already savd off the netcdfs, so can just open them below 

In [18]:
# save off anoms for SEA plots 
with dask.diagnostics.ProgressBar():
    if var == 'tmp':
        A_mon_anom_Glob.tmp.to_netcdf(f'{path}A_mon_anom_Glob_{var}.nc')
    elif var == 'pr':
        K_mon_anom_Glob.pr.to_netcdf(f'{path}K_mon_anom_Glob_{var}.nc')

AttributeError: 'NoneType' object has no attribute 'lock_acquire'

distributed.nanny - WARNING - Restarting worker


In [16]:
# save off anoms for SEA plots 
with dask.diagnostics.ProgressBar():
    if var == 'tmp':
        S_mon_anom_Glob.tmp.to_netcdf(f'{path}S_mon_anom_Glob_{var}.nc')
        A_mon_anom_Glob.tmp.to_netcdf(f'{path}A_mon_anom_Glob_{var}.nc')
        E_mon_anom_Glob.tmp.to_netcdf(f'{path}E_mon_anom_Glob_{var}.nc')
        P_mon_anom_Glob.tmp.to_netcdf(f'{path}P_mon_anom_Glob_{var}.nc')
    elif var == 'pr':
#         S_mon_anom_Glob.pr.to_netcdf(f'{path}S_mon_anom_Glob_{var}.nc')
        A_mon_anom_Glob.pr.to_netcdf(f'{path}A_mon_anom_Glob_{var}.nc')
        E_mon_anom_Glob.pr.to_netcdf(f'{path}E_mon_anom_Glob_{var}.nc')
        P_mon_anom_Glob.pr.to_netcdf(f'{path}P_mon_anom_Glob_{var}.nc')

distributed.nanny - WARNING - Restarting worker
distributed.nanny - WARNING - Restarting worker
distributed.nanny - WARNING - Restarting worker
distributed.nanny - WARNING - Restarting worker


KilledWorker: ("('getitem-mean_chunk-c5bacdaeeb957577727c52a76b2f60ca', 10, 0, 0, 0)", <WorkerState 'tcp://127.0.0.1:42287', name: 1, memory: 0, processing: 302>)

In [22]:
# open anoms 
K_mon_anom_Glob = xr.open_dataset(f'{path}K_mon_anom_Glob_{var}.nc')
S_mon_anom_Glob = xr.open_dataset(f'{path}S_mon_anom_Glob_{var}.nc')
A_mon_anom_Glob = xr.open_dataset(f'{path}A_mon_anom_Glob_{var}.nc')
E_mon_anom_Glob = xr.open_dataset(f'{path}E_mon_anom_Glob_{var}.nc')
P_mon_anom_Glob = xr.open_dataset(f'{path}P_mon_anom_Glob_{var}.nc')

FileNotFoundError: [Errno 2] No such file or directory: b'/g/data/w48/kb6999/Reanalysis/20CR_SEA_data_for_plots/K_mon_anom_Glob_tmp.nc'

In [19]:
times = np.arange(-60,61)
# reset all the times so 0 corresponds to the eruption year and month
K_mon_anom_Glob['time'] = times
S_mon_anom_Glob['time'] = times
A_mon_anom_Glob['time'] = times
E_mon_anom_Glob['time'] = times 
P_mon_anom_Glob['time'] = times

# delete the months dimension (but not the time)
if 'month' in P_mon_anom_Glob:
    del K_mon_anom_Glob['month']
    del S_mon_anom_Glob['month']
    del A_mon_anom_Glob['month']
    del E_mon_anom_Glob['month']
    del P_mon_anom_Glob['month']

In [21]:
# import land fraction data
if var =='tmp':
    landfrac_ds = xr.open_dataset('/g/data/w48/kb6999/Reanalysis/20CR_data_netcdfs/land_20CR.nc')
    landmask = landfrac_ds
    
if var =='pr':
    landfrac_ds = xr.open_dataset('/g/data/w48/kb6999/Reanalysis/20CR_data_netcdfs/land_pr_20CR.nc')
    landmask = landfrac_ds

## krakatoa 

In [26]:
#  select out each region for each volvnao
K_mon_SH = K_mon_anom_Glob.sel(lat=slice(0,-90)) 
K_mon_Aus = K_mon_anom_Glob.sel(lat=slice(-10,-50), lon=slice(110,160)).where(landmask==1, drop=True)
K_mon_EA = K_mon_anom_Glob.sel(lat=slice(-10,-50), lon=slice(140,155)).where(landmask==1, drop=True)

In [27]:
K_mon_EA

<xarray.Dataset>
Dimensions:  (lat: 44, lon: 19, member: 79, time: 121)
Coordinates:
  * lat      (lat) float64 -10.18 -11.58 -12.28 -12.98 ... -41.75 -42.46 -43.16
  * lon      (lon) float64 140.6 141.3 142.0 142.7 ... 151.1 151.8 152.6 153.3
  * time     (time) int64 -60 -59 -58 -57 -56 -55 -54 ... 54 55 56 57 58 59 60
  * member   (member) int64 1 2 3 4 5 6 7 8 9 10 ... 72 73 74 75 76 77 78 79 80
Data variables:
    *empty*

In [28]:
# calculate the monthly lat lon mean for monthly anomalies
K_llm_Glob = K_mon_anom_Glob.mean(dim=('lat','lon'))
K_llm_SH = K_mon_SH.mean(dim=('lat','lon'))
K_llm_Aus = K_mon_Aus.mean(dim=('lat','lon'))
K_llm_EA = K_mon_EA.mean(dim=('lat','lon'))
K_llm_Aus

<xarray.Dataset>
Dimensions:  (member: 79, time: 121)
Coordinates:
  * time     (time) int64 -60 -59 -58 -57 -56 -55 -54 ... 54 55 56 57 58 59 60
  * member   (member) int64 1 2 3 4 5 6 7 8 9 10 ... 72 73 74 75 76 77 78 79 80
Data variables:
    *empty*

In [29]:
# save off llm for SEA plots krakatoa
with dask.diagnostics.ProgressBar():
    if var == 'tmp':
        K_llm_Glob.tmp.to_netcdf(f'{path}K_llm_Glob_{var}.nc')
        K_llm_SH.tmp.to_netcdf(f'{path}K_llm_SH_{var}.nc')
        K_llm_Aus.tmp.to_netcdf(f'{path}K_llm_Aus_{var}.nc')
        K_llm_EA.tmp.to_netcdf(f'{path}K_llm_EA_{var}.nc')
    elif var == 'pr':
        K_llm_Glob.pr.to_netcdf(f'{path}K_llm_Glob_{var}.nc')
        K_llm_SH.pr.to_netcdf(f'{path}K_llm_SH_{var}.nc')
        K_llm_Aus.pr.to_netcdf(f'{path}K_llm_Aus_{var}.nc')
        K_llm_EA.pr.to_netcdf(f'{path}K_llm_EA_{var}.nc')

distributed.nanny - WARNING - Restarting worker
distributed.nanny - WARNING - Restarting worker
distributed.nanny - WARNING - Restarting worker
distributed.nanny - WARNING - Restarting worker
distributed.nanny - WARNING - Restarting worker


KilledWorker: ('open_dataset-14574e1cf8a655bff93caa135ce4a9cdTMP-5c75c89ab1e7b0e4a41f50543cd2e887', <WorkerState 'tcp://127.0.0.1:45761', name: 1, memory: 0, processing: 500>)

In [ ]:
# calculate the monthly model mean for monthly anomalies
K_mmm_Glob = K_mon_anom_Glob.mean(dim='member').mean(dim=('lat','lon'))
K_mmm_SH = K_mon_SH.mean(dim='member').mean(dim=('lat','lon'))
K_mmm_Aus = K_mon_Aus.mean(dim='member').mean(dim=('lat','lon'))
K_mmm_EA = K_mon_EA.mean(dim='member').mean(dim=('lat','lon'))
K_mmm_Aus

In [ ]:
# save off llm mmm for SEA plots krakatoa
with dask.diagnostics.ProgressBar():
    if var == 'tmp':
        K_mmm_Glob.tmp.to_netcdf(f'{path}K_mmm_Glob_{var}.nc')
        K_mmm_SH.tmp.to_netcdf(f'{path}K_mmm_SH_{var}.nc')
        K_mmm_Aus.tmp.to_netcdf(f'{path}K_mmm_Aus_{var}.nc')
        K_mmm_EA.tmp.to_netcdf(f'{path}K_mmm_EA_{var}.nc')
    elif var == 'pr':
        K_mmm_Glob.pr.to_netcdf(f'{path}K_mmm_Glob_{var}.nc')
        K_mmm_SH.pr.to_netcdf(f'{path}K_mmm_SH_{var}.nc')
        K_mmm_Aus.pr.to_netcdf(f'{path}K_mmm_Aus_{var}.nc')
        K_mmm_EA.pr.to_netcdf(f'{path}K_mmm_EA_{var}.nc')

## santa maria

In [12]:
#  select out each region for eac
S_mon_SH = S_mon_anom_Glob.sel(lat=slice(-90,0))
S_mon_Aus = S_mon_anom_Glob.sel(lat=slice(-50,-10), lon=slice(110,160)).where(landmask.data==1, drop=True)
S_mon_EA = S_mon_anom_Glob.sel(lat=slice(-50,-10), lon=slice(140,155)).where(landmask.data==1, drop=True)

In [13]:
# calculate the monthly member mean for monthly anomalies
S_llm_Glob = S_mon_anom_Glob.mean(dim=('lat','lon'))
S_llm_SH = S_mon_SH.mean(dim=('lat','lon'))
S_llm_Aus = S_mon_Aus.mean(dim=('lat','lon'))
S_llm_EA = S_mon_EA.mean(dim=('lat','lon'))
S_llm_Aus

<xarray.Dataset>
Dimensions:  (model: 43, time: 121)
Coordinates:
  * time     (time) int64 -60 -59 -58 -57 -56 -55 -54 ... 54 55 56 57 58 59 60
  * model    (model) object 'ACCESS-CM2' 'ACCESS-ESM1-5' ... 'TaiESM1'
Data variables:
    tas      (model, time) float64 0.715 -0.03689 0.536 ... 0.7583 0.1736 -1.809

In [14]:
# save off llm for SEA plots krakatoa
with dask.diagnostics.ProgressBar():
    if var == 'tmp':
        S_llm_Glob.tmp.to_netcdf(f'{path}S_llm_Glob_{var}.nc')
        S_llm_SH.tmp.to_netcdf(f'{path}S_llm_SH_{var}.nc')
        S_llm_Aus.tmp.to_netcdf(f'{path}S_llm_Aus_{var}.nc')
        S_llm_EA.tmp.to_netcdf(f'{path}S_llm_EA_{var}.nc')
    elif var == 'pr':
        S_llm_Glob.pr.to_netcdf(f'{path}S_llm_Glob_{var}.nc')
        S_llm_SH.pr.to_netcdf(f'{path}S_llm_SH_{var}.nc')
        S_llm_Aus.pr.to_netcdf(f'{path}S_llm_Aus_{var}.nc')
        S_llm_EA.pr.to_netcdf(f'{path}S_llm_EA_{var}.nc')

In [14]:
# calculate the monthly model mean for monthly anomalies
S_mmm_Glob = S_mon_anom_Glob.mean(dim='member').mean(dim=('lat','lon'))
S_mmm_SH = S_mon_SH.mean(dim='member').mean(dim=('lat','lon'))
S_mmm_Aus = S_mon_Aus.mean(dim='member').mean(dim=('lat','lon'))
S_mmm_EA = S_mon_EA.mean(dim='member').mean(dim=('lat','lon'))
S_mmm_Aus

<xarray.Dataset>
Dimensions:  (time: 121)
Coordinates:
  * time     (time) int64 -60 -59 -58 -57 -56 -55 -54 ... 54 55 56 57 58 59 60
Data variables:
    pr       (time) float64 -0.01563 0.02718 0.0006509 ... 0.07275 0.1287

In [15]:
# save off llm mmm for SEA plots santa maria
with dask.diagnostics.ProgressBar():
    if var == 'tmp':
        S_mmm_Glob.tmp.to_netcdf(f'{path}S_mmm_Glob_{var}.nc')
        S_mmm_SH.tmp.to_netcdf(f'{path}S_mmm_SH_{var}.nc')
        S_mmm_Aus.tmp.to_netcdf(f'{path}S_mmm_Aus_{var}.nc')
        S_mmm_EA.tmp.to_netcdf(f'{path}S_mmm_EA_{var}.nc')
    elif var == 'pr':
        S_mmm_Glob.pr.to_netcdf(f'{path}S_mmm_Glob_{var}.nc')
        S_mmm_SH.pr.to_netcdf(f'{path}S_mmm_SH_{var}.nc')
        S_mmm_Aus.pr.to_netcdf(f'{path}S_mmm_Aus_{var}.nc')
        S_mmm_EA.pr.to_netcdf(f'{path}S_mmm_EA_{var}.nc')

## agung

In [12]:
#  select out each region for eac
A_mon_SH = A_mon_anom_Glob.sel(lat=slice(-90,0))
A_mon_Aus = A_mon_anom_Glob.sel(lat=slice(-50,-10), lon=slice(110,160)).where(landmask.data==1, drop=True)
A_mon_EA = A_mon_anom_Glob.sel(lat=slice(-50,-10), lon=slice(140,155)).where(landmask.data==1, drop=True)

In [13]:
# calculate the monthly model mean for monthly anomalies
A_llm_Glob = A_mon_anom_Glob.mean(dim=('lat','lon'))
A_llm_SH = A_mon_SH.mean(dim=('lat','lon'))
A_llm_Aus = A_mon_Aus.mean(dim=('lat','lon'))
A_llm_EA = A_mon_EA.mean(dim=('lat','lon'))
A_llm_Aus

<xarray.Dataset>
Dimensions:  (model: 43, time: 121)
Coordinates:
  * time     (time) int64 -60 -59 -58 -57 -56 -55 -54 ... 54 55 56 57 58 59 60
  * model    (model) object 'ACCESS-CM2' 'ACCESS-ESM1-5' ... 'TaiESM1'
Data variables:
    tas      (model, time) float64 0.3763 0.5238 0.2524 ... -1.348 0.1116

In [14]:
# save off llm for SEA plots krakatoa
with dask.diagnostics.ProgressBar():
    if var == 'tmp':
        A_llm_Glob.tmp.to_netcdf(f'{path}A_llm_Glob_{var}.nc')
        A_llm_SH.tmp.to_netcdf(f'{path}A_llm_SH_{var}.nc')
        A_llm_Aus.tmp.to_netcdf(f'{path}A_llm_Aus_{var}.nc')
        A_llm_EA.tmp.to_netcdf(f'{path}A_llm_EA_{var}.nc')
    elif var == 'pr':
        A_llm_Glob.pr.to_netcdf(f'{path}A_llm_Glob_{var}.nc')
        A_llm_SH.pr.to_netcdf(f'{path}A_llm_SH_{var}.nc')
        A_llm_Aus.pr.to_netcdf(f'{path}A_llm_Aus_{var}.nc')
        A_llm_EA.pr.to_netcdf(f'{path}A_llm_EA_{var}.nc')

In [13]:
# calculate the monthly member mean for monthly anomalies
A_mmm_Glob = A_mon_anom_Glob.mean(dim='member').mean(dim=('lat','lon'))
A_mmm_SH = A_mon_SH.mean(dim='member').mean(dim=('lat','lon'))
A_mmm_Aus = A_mon_Aus.mean(dim='member').mean(dim=('lat','lon'))
A_mmm_EA = A_mon_EA.mean(dim='member').mean(dim=('lat','lon'))
A_mmm_Aus

<xarray.Dataset>
Dimensions:  (time: 121)
Coordinates:
  * time     (time) int64 -60 -59 -58 -57 -56 -55 -54 ... 54 55 56 57 58 59 60
Data variables:
    pr       (time) float64 0.0002863 0.07069 0.03208 ... -0.1756 -0.08233

In [14]:
# save off llm mmm for SEA plots agung
with dask.diagnostics.ProgressBar():
    if var == 'tmp':
        A_mmm_Glob.tmp.to_netcdf(f'{path}A_mmm_Glob_{var}.nc')
        A_mmm_SH.tmp.to_netcdf(f'{path}A_mmm_SH_{var}.nc')
        A_mmm_Aus.tmp.to_netcdf(f'{path}A_mmm_Aus_{var}.nc')
        A_mmm_EA.tmp.to_netcdf(f'{path}A_mmm_EA_{var}.nc')
    elif var == 'pr':
        A_mmm_Glob.pr.to_netcdf(f'{path}A_mmm_Glob_{var}.nc')
        A_mmm_SH.pr.to_netcdf(f'{path}A_mmm_SH_{var}.nc')
        A_mmm_Aus.pr.to_netcdf(f'{path}A_mmm_Aus_{var}.nc')
        A_mmm_EA.pr.to_netcdf(f'{path}A_mmm_EA_{var}.nc')

## el chichon

In [12]:
#  select out each region for eac
E_mon_SH = E_mon_anom_Glob.sel(lat=slice(-90,0))
E_mon_Aus = E_mon_anom_Glob.sel(lat=slice(-50,-10), lon=slice(110,160)).where(landmask.data==1, drop=True)
E_mon_EA = E_mon_anom_Glob.sel(lat=slice(-50,-10), lon=slice(140,155)).where(landmask.data==1, drop=True)

In [13]:
# calculate the monthly model mean for monthly anomalies
E_llm_Glob = E_mon_anom_Glob.mean(dim=('lat','lon'))
E_llm_SH = E_mon_SH.mean(dim=('lat','lon'))
E_llm_Aus = E_mon_Aus.mean(dim=('lat','lon'))
E_llm_EA = E_mon_EA.mean(dim=('lat','lon'))
E_llm_Aus

<xarray.Dataset>
Dimensions:  (model: 43, time: 121)
Coordinates:
  * time     (time) int64 -60 -59 -58 -57 -56 -55 -54 ... 54 55 56 57 58 59 60
  * model    (model) object 'ACCESS-CM2' 'ACCESS-ESM1-5' ... 'TaiESM1'
Data variables:
    tas      (model, time) float64 0.8758 1.41 0.1287 ... -0.5571 -0.7952 0.2338

In [14]:
# save off llm for SEA plots krakatoa
with dask.diagnostics.ProgressBar():
    if var == 'tmp':
        E_llm_Glob.tmp.to_netcdf(f'{path}E_llm_Glob_{var}.nc')
        E_llm_SH.tmp.to_netcdf(f'{path}E_llm_SH_{var}.nc')
        E_llm_Aus.tmp.to_netcdf(f'{path}E_llm_Aus_{var}.nc')
        E_llm_EA.tmp.to_netcdf(f'{path}E_llm_EA_{var}.nc')
    elif var == 'pr':
        E_llm_Glob.pr.to_netcdf(f'{path}E_llm_Glob_{var}.nc')
        E_llm_SH.pr.to_netcdf(f'{path}E_llm_SH_{var}.nc')
        E_llm_Aus.pr.to_netcdf(f'{path}E_llm_Aus_{var}.nc')
        E_llm_EA.pr.to_netcdf(f'{path}E_llm_EA_{var}.nc')

In [13]:
# calculate the monthly model mean for monthly anomalies
E_mmm_Glob = E_mon_anom_Glob.mean(dim='member').mean(dim=('lat','lon'))
E_mmm_SH = E_mon_SH.mean(dim='member').mean(dim=('lat','lon'))
E_mmm_Aus = E_mon_Aus.mean(dim='member').mean(dim=('lat','lon'))
E_mmm_EA = E_mon_EA.mean(dim='member').mean(dim=('lat','lon'))
E_mmm_Aus

<xarray.Dataset>
Dimensions:  (time: 121)
Coordinates:
  * time     (time) int64 -60 -59 -58 -57 -56 -55 -54 ... 54 55 56 57 58 59 60
Data variables:
    pr       (time) float64 -0.1508 -0.05844 0.004157 ... -0.2783 0.3912 -0.0385

In [14]:
# save off llm mmm for SEA plots el chichon
with dask.diagnostics.ProgressBar():
    if var == 'tmp':
        E_mmm_Glob.tmp.to_netcdf(f'{path}E_mmm_Glob_{var}.nc')
        E_mmm_SH.tmp.to_netcdf(f'{path}E_mmm_SH_{var}.nc')
        E_mmm_Aus.tmp.to_netcdf(f'{path}E_mmm_Aus_{var}.nc')
        E_mmm_EA.tmp.to_netcdf(f'{path}E_mmm_EA_{var}.nc')
    elif var == 'pr':
        E_mmm_Glob.pr.to_netcdf(f'{path}E_mmm_Glob_{var}.nc')
        E_mmm_SH.pr.to_netcdf(f'{path}E_mmm_SH_{var}.nc')
        E_mmm_Aus.pr.to_netcdf(f'{path}E_mmm_Aus_{var}.nc')
        E_mmm_EA.pr.to_netcdf(f'{path}E_mmm_EA_{var}.nc')


## pinatubo

In [8]:
#  select out each region for eac
P_mon_SH = P_mon_anom_Glob.sel(lat=slice(-90,0))
P_mon_Aus = P_mon_anom_Glob.sel(lat=slice(-50,-10), lon=slice(110,160)).where(landmask.data==1, drop=True)
P_mon_EA = P_mon_anom_Glob.sel(lat=slice(-50,-10), lon=slice(140,155)).where(landmask.data==1, drop=True)

In [9]:
# calculate the monthly model mean for monthly anomalies
P_llm_Glob = P_mon_anom_Glob.mean(dim=('lat','lon'))
P_llm_SH = P_mon_SH.mean(dim=('lat','lon'))
P_llm_Aus = P_mon_Aus.mean(dim=('lat','lon'))
P_llm_EA = P_mon_EA.mean(dim=('lat','lon'))
P_llm_Aus

<xarray.Dataset>
Dimensions:  (model: 43, time: 121)
Coordinates:
  * time     (time) int64 -60 -59 -58 -57 -56 -55 -54 ... 54 55 56 57 58 59 60
  * model    (model) object 'ACCESS-CM2' 'ACCESS-ESM1-5' ... 'TaiESM1'
Data variables:
    tas      (model, time) float64 -1.123 0.4511 -0.5525 ... -1.591 -1.09 -0.436

In [10]:
# save off llm for SEA plots krakatoa
with dask.diagnostics.ProgressBar():
    if var == 'tmp':
        P_llm_Glob.tmp.to_netcdf(f'{path}P_llm_Glob_{var}.nc')
        P_llm_SH.tmp.to_netcdf(f'{path}P_llm_SH_{var}.nc')
        P_llm_Aus.tmp.to_netcdf(f'{path}P_llm_Aus_{var}.nc')
        P_llm_EA.tmp.to_netcdf(f'{path}P_llm_EA_{var}.nc')
    elif var == 'pr':
        P_llm_Glob.pr.to_netcdf(f'{path}P_llm_Glob_{var}.nc')
        P_llm_SH.pr.to_netcdf(f'{path}P_llm_SH_{var}.nc')
        P_llm_Aus.pr.to_netcdf(f'{path}P_llm_Aus_{var}.nc')
        P_llm_EA.pr.to_netcdf(f'{path}P_llm_EA_{var}.nc')

In [13]:
# calculate the monthly model mean for monthly anomalies
P_mmm_Glob = P_mon_anom_Glob.mean(dim='member').mean(dim=('lat','lon'))
P_mmm_SH = P_mon_SH.mean(dim='member').mean(dim=('lat','lon'))
P_mmm_Aus = P_mon_Aus.mean(dim='member').mean(dim=('lat','lon'))
P_mmm_EA = P_mon_EA.mean(dim='member').mean(dim=('lat','lon'))
P_mmm_Aus

<xarray.Dataset>
Dimensions:  (time: 121)
Coordinates:
  * time     (time) int64 -60 -59 -58 -57 -56 -55 -54 ... 54 55 56 57 58 59 60
Data variables:
    pr       (time) float64 0.194 0.08357 0.2175 ... 0.002293 0.02728 -0.02611

In [14]:
# save off llm mmm for SEA plots krakatoa
with dask.diagnostics.ProgressBar():
    if var == 'tmp':
        P_mmm_Glob.tmp.to_netcdf(f'{path}P_mmm_Glob_{var}.nc')
        P_mmm_SH.tmp.to_netcdf(f'{path}P_mmm_SH_{var}.nc')
        P_mmm_Aus.tmp.to_netcdf(f'{path}P_mmm_Aus_{var}.nc')
        P_mmm_EA.tmp.to_netcdf(f'{path}P_mmm_EA_{var}.nc')
    elif var == 'pr':
        P_mmm_Glob.pr.to_netcdf(f'{path}P_mmm_Glob_{var}.nc')
        P_mmm_SH.pr.to_netcdf(f'{path}P_mmm_SH_{var}.nc')
        P_mmm_Aus.pr.to_netcdf(f'{path}P_mmm_Aus_{var}.nc')
        P_mmm_EA.pr.to_netcdf(f'{path}P_mmm_EA_{var}.nc')

In [ ]:
# # select out each region
# composite_mon_SH = composite_mon_Glob.sel(lat=slice(-90,0))
# composite_mon_Aus = composite_mon_Glob.sel(lat=slice(-50,-10), lon=slice(110,160)).where(landmask.data==1, drop=True)
# composite_mon_EA = composite_mon_Glob.sel(lat=slice(-50,-10), lon=slice(140,155)).where(landmask.data==1, drop=True)